## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [1]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 18 enero

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * `estado_vacunacion_.csv` , última versión del mismo con el acumulado de todos los informes publicados
  * `estado_vacunacion_yearmonthday.csv`, con la versión del día en cuestión, solo con la información de ese día

In [2]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=2,day=19)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en red en formato ods

In [3]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_GIV_comunicacion_{}.ods'
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [4]:
enlace

'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210324.ods'

In [5]:
try:
    datos_row = pd.read_excel(enlace)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210324.ods


Le echamos un vistazo para asegurar que ha ido todo bien

In [6]:
datos_row

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Dosis entregadas AstraZeneca (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas vacunadas(pauta completada),Fecha de la última vacuna registrada (2)
0,Andalucía,999270,111200,361600,1472070,1258375,0.854834,455368,2021-03-23
1,Aragón,187515,21700,56800,266015,211426,0.794790,74363,2021-03-23
2,Asturias,182255,20700,43600,246555,192737,0.781720,76050,2021-03-23
3,Baleares,99810,11100,49900,160810,125552,0.780747,36060,2021-03-23
4,Canarias,209400,22900,93000,325300,248101,0.762684,86926,2021-03-23
5,Cantabria,88485,9700,24800,122985,94153,0.765565,39010,2021-03-23
6,Castilla y Leon,413895,47700,102400,563995,441559,0.782913,163995,2021-03-23
7,Castilla La Mancha,270135,31000,87400,388535,278619,0.717101,98356,2021-03-23
8,Cataluña,923750,105400,332500,1361650,1027370,0.754504,356766,2021-03-23
9,C. Valenciana,518500,57800,216100,792400,590395,0.745072,215050,2021-03-23


..hacemos una copia para hacerle las transformaciones necesarias:

In [7]:
datos_hoy = datos_row.copy()

Primero renombramos columnas, añadimos luego una columna con la fecha del informe `date_pub`, y finalmente añadimos fuente y origen de datos para asegurar trazabilidad de los datos:

In [8]:
datos_hoy.columns=['ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas','Dosis administradas','% sobre entregadas','Total pauta completada','Fecha de la ultima vacuna registrada']

In [9]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de `Fecha de la ultima vacuna registrada` para que aparezca como día/mes/año sin añadir ceros de pad..de ahi el signo menos en el caso de dia y mes (%-d/%-m). La excepción se añade para la línea de Totales, que no lleva fecha asociada:

In [10]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [11]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

..y ordenamos..

In [12]:
datos_hoy

,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,date_pub,source_name,source
0,Andalucía,999270,111200,361600,1472070,1258375,0.854834,455368,23/3/21,24/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,Aragón,187515,21700,56800,266015,211426,0.794790,74363,23/3/21,24/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,Asturias,182255,20700,43600,246555,192737,0.781720,76050,23/3/21,24/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,Baleares,99810,11100,49900,160810,125552,0.780747,36060,23/3/21,24/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,Canarias,209400,22900,93000,325300,248101,0.762684,86926,23/3/21,24/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,Cantabria,88485,9700,24800,122985,94153,0.765565,39010,23/3/21,24/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,Castilla y Leon,413895,47700,102400,563995,441559,0.782913,163995,23/3/21,24/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,Castilla La Mancha,270135,31000,87400,388535,278619,0.717101,98356,23/3/21,24/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,Cataluña,923750,105400,332500,1361650,1027370,0.754504,356766,23/3/21,24/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,C. Valenciana,518500,57800,216100,792400,590395,0.745072,215050,23/3/21,24/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [13]:
datos_hoy=datos_hoy[['date_pub','ccaa','Dosis entregadas Pfizer',
                    'Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas',
                    'Dosis administradas','% sobre entregadas',
                    'Total pauta completada',
                    'Fecha de la ultima vacuna registrada','source_name','source']]

Hemos visto algún informe en el que se añaden espacios al nombre de Comunidad, nos aseguramos aquí de eliminarlos en caso de que vuelvan a aparecer:

In [14]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

In [15]:
datos_hoy['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla',
       'Fuerzas Armadas', 'Totales'], dtype=object)

In [16]:
datos_hoy

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,source_name,source
0,24/3/21,Andalucía,999270,111200,361600,1472070,1258375,0.854834,455368,23/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,24/3/21,Aragón,187515,21700,56800,266015,211426,0.794790,74363,23/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,24/3/21,Asturias,182255,20700,43600,246555,192737,0.781720,76050,23/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,24/3/21,Baleares,99810,11100,49900,160810,125552,0.780747,36060,23/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,24/3/21,Canarias,209400,22900,93000,325300,248101,0.762684,86926,23/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,24/3/21,Cantabria,88485,9700,24800,122985,94153,0.765565,39010,23/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,24/3/21,Castilla y Leon,413895,47700,102400,563995,441559,0.782913,163995,23/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,24/3/21,Castilla La Mancha,270135,31000,87400,388535,278619,0.717101,98356,23/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,24/3/21,Cataluña,923750,105400,332500,1361650,1027370,0.754504,356766,23/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,24/3/21,C. Valenciana,518500,57800,216100,792400,590395,0.745072,215050,23/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [17]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [18]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [19]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,23/3/21,Navarra,76785.0,9700.0,26800.0,113285,100046,0.883135,31120.0,NaN,22/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1096,23/3/21,País Vasco,238500.0,30800.0,90200.0,359500,277455,0.771780,75513.0,NaN,22/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1097,23/3/21,Ceuta,5490.0,900.0,3500.0,9890,9432,0.953691,2542.0,NaN,17/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1098,23/3/21,Melilla,4320.0,800.0,3500.0,8620,8937,1.036775,2387.0,NaN,22/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [20]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [21]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Dosis entregadas Pfizer',
       'Dosis entregadas Moderna', 'Dosis entregadas AstraZeneca',
       'Dosis entregadas', 'Dosis administradas', '% sobre entregadas',
       'Total pauta completada', 'Última fecha de actualización de datos',
       'Fecha de la ultima vacuna registrada', 'source_name', 'source'],
      dtype='object')

In [22]:
datos_acumulados = datos_acumulados[['date_pub', 'ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna',
                                     'Dosis entregadas AstraZeneca',
                                     'Dosis entregadas', 'Dosis administradas','% sobre entregadas',
                                     'Total pauta completada','Última fecha de actualización de datos',
                                     'Fecha de la ultima vacuna registrada','source_name', 'source']]

In [23]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,24/3/21,País Vasco,275940.0,30800.0,94900.0,401640,289725,0.721355,78403.0,NaN,23/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1117,24/3/21,Ceuta,6660.0,900.0,3700.0,11260,9457,0.839876,2566.0,NaN,23/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1118,24/3/21,Melilla,6660.0,800.0,3700.0,11160,9074,0.813082,2387.0,NaN,23/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1119,24/3/21,Fuerzas Armadas,19500.0,0.0,33100.0,52600,15777,0.299943,4658.0,NaN,23/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [24]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales',
       'Fuerzas Armadas'], dtype=object)

y volvemos a subirlo..

In [25]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)